In [1]:
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
import pandexo.engine.justdoit as jdi # THIS IS THE HOLY GRAIL OF PANDEXO
import os

import pandexo.engine.justplotit as jpi
import pickle as pk

In [2]:
print(os.environ['pandeia_refdata'] )
import pandeia.engine
print(pandeia.engine.__version__)

/Users/coffey/Downloads/kipping/data/pandeia
3.0


### Loading blank exo dictionary

In [3]:
exo_dict = jdi.load_exo_dict()
print(exo_dict.keys())
#print(exo_dict['star']['w_unit'])

dict_keys(['calculation', 'url', 'url_err', 'star', 'planet', 'observation'])


### Edit planet and star properties

In [4]:
exo_dict['observation']['sat_level'] = 80    #saturation level in percent of full well 
exo_dict['observation']['sat_unit'] = '%'
exo_dict['observation']['noccultations'] = 1 #number of transits 
exo_dict['observation']['R'] = None           #fixed binning. I usually suggest ZERO binning.. you can always bin later 
                                             #without having to redo the calcualtion
exo_dict['observation']['baseline_unit'] = 'total'  #Defines how you specify out of transit observing time
                                                    #'frac' : fraction of time in transit versus out = in/out 
                                                    #'total' : total observing time (seconds)
exo_dict['observation']['baseline'] = 3.0*60.0*60.0 #in accordance with what was specified above (total observing time)

exo_dict['observation']['noise_floor'] = 14  #this can be a fixed level or it can be a filepath 
                                             #to a wavelength dependent noise floor solution (units are ppm)

In [5]:
#OPTION 1 get start from database
exo_dict['star']['type'] = 'phoenix'        #phoenix or user (if you have your own)
exo_dict['star']['mag'] = 11.354            #magnitude of the system
exo_dict['star']['ref_wave'] = 1.25         #For J mag = 1.25, H = 1.6, K =2.22.. etc (all in micron)
exo_dict['star']['temp'] = 2566             #in K 
exo_dict['star']['metal'] = -1.4            # as log Fe/H
exo_dict['star']['logg'] = 5.276            #log surface gravity cgs

In [6]:
exo_dict['planet']['type'] = 'constant'                  #tells pandexo you want a fixed transit depth

exo_dict['planet']['transit_duration'] = 0.6*60.0*60.0   #transit duration 
exo_dict['planet']['radius'] = 0.0996
exo_dict['planet']['r_unit'] = 'R_jup'                   #Any unit of distance in accordance with astropy.units can be added here
exo_dict['star']['radius'] = 0.119
exo_dict['star']['r_unit'] = 'R_sun'

exo_dict['planet']['td_unit'] = 's'                      #Any unit of time in accordance with astropy.units can be added
exo_dict['planet']['w_unit'] = 'um'                      #other options include "um","nm" ,"Angs", "sec" (for phase curves)
exo_dict['planet']['f_unit'] = 'rp^2/r*^2'               #other options are 'fp/f*' 

In [7]:
result = jdi.run_pandexo(exo_dict, ['NIRSpec Prism'], output_file='trappist_pandexo.p')

Running Single Case for: NIRSpec Prism
Optimization Reqested: Computing Duty Cycle
Finished Duty Cycle Calc
Starting Out of Transit Simulation
End out of Transit
Starting In Transit Simulation
End In Transit


### Plotting results

In [8]:
#load in output from run
out = pk.load(open('trappist_pandexo.p','rb'))
#for a single run
x, y, e = jpi.jwst_1d_spec(out, x_range=[0.6,5.3])

x100, y100, e100 = jpi.jwst_1d_spec(out, R = 100, x_range=[0.6,5.3])

Loading BokehJS ...

Loading BokehJS ...

## Saving errors to text file

In [9]:
# Saving the array in a text file
pandexo_hacked_errs = np.column_stack((x[0], y[0]*1e6, e[0]*1e6))
np.savetxt("./pandexo_output/pandexo_errs.txt", pandexo_hacked_errs)

pandexo_hacked_errs_100 = np.column_stack((x100[0], y100[0]*1e6, e100[0]*1e6))
np.savetxt("./pandexo_output/pandexo_errs_100.txt", pandexo_hacked_errs_100)

In [10]:
err = jpi.jwst_noise(out)